In [100]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
import seaborn as sns

# URLs de los datos
urls = {
    "nino12": "https://psl.noaa.gov/gcos_wgsp/Timeseries/Data/nino12.long.anom.data",
    "nino3": "https://psl.noaa.gov/gcos_wgsp/Timeseries/Data/nino3.long.anom.data",
    "nino4": "https://psl.noaa.gov/gcos_wgsp/Timeseries/Data/nino4.long.anom.data",
    "nino34": "https://psl.noaa.gov/gcos_wgsp/Timeseries/Data/nino34.long.anom.data"
}

# Dataframe para almacenar los datos combinados
combined_df = pd.DataFrame()

# Leer y transformar cada archivo
for name, url in urls.items():
    # Leer el archivo
    df = pd.read_csv(url, delim_whitespace=True, header=None, skiprows=1)

    # Asignar nombres de columnas
    column_names = ["Year", "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    df.columns = column_names

    # Crear una lista para almacenar las filas transformadas
    rows = []

    # Iterar sobre cada fila del dataframe original
    for _, row in df.iterrows():
        year = row["Year"]
        try:
            year = int(year)
        except ValueError:
            continue
        for month_idx, month_name in enumerate(column_names[1:], start=1):
            anomaly = row[month_name]
            if anomaly != -99.99:
                date = pd.to_datetime(f"{year}-{month_idx:02d}-01")
                rows.append({"Date": date, name: anomaly})

    # Convertir la lista de filas a un nuevo dataframe
    transformed_df = pd.DataFrame(rows)

    # Combinar con el dataframe principal
    if combined_df.empty:
        combined_df = transformed_df
    else:
        combined_df = pd.merge(combined_df, transformed_df, on="Date", how="outer")

# Ordenar por fecha
combined_df = combined_df.sort_values("Date").reset_index(drop=True)

# Mostrar el dataframe combinado
print("Combined Dataframe")
print(combined_df.head())


Combined Dataframe
        Date nino12  nino3  nino4 nino34
0 1870-01-01  -1.50  -1.35  -0.48  -1.00
1 1870-02-01  -0.96  -1.20  -1.16  -1.20
2 1870-03-01  -0.65  -0.76  -1.01  -0.83
3 1870-04-01  -0.32  -0.78   -1.0  -0.81
4 1870-05-01  -0.64  -1.15  -1.08  -1.27


In [101]:
# Guardarmos el DataFrame en un archivo CSV
combined_df.to_csv('Enso.csv', index=False)



In [102]:
df = combined_df
df

,Date,nino12,nino3,nino4,nino34
0,1870-01-01,-1.50,-1.35,-0.48,-1.00
1,1870-02-01,-0.96,-1.20,-1.16,-1.20
2,1870-03-01,-0.65,-0.76,-1.01,-0.83
3,1870-04-01,-0.32,-0.78,-1.0,-0.81
4,1870-05-01,-0.64,-1.15,-1.08,-1.27
...,...,...,...,...,...
1848,2024-01-01,1.14,1.74,1.40,1.71
1849,2024-02-01,1.08,1.45,1.21,1.47
1850,2024-03-01,-0.06,0.84,0.99,1.1
1851,2024-04-01,-0.11,0.7,0.85,0.93


In [103]:
# Convertir las columnas a valores numéricos (si hay valores no numéricos, se convierten a NaN)
df[['nino12', 'nino3', 'nino4', 'nino34']] = df[['nino12', 'nino3', 'nino4', 'nino34']].apply(pd.to_numeric, errors='coerce')

# Calcular el valor mínimo y máximo de las columnas de anomalías
min_value = df[['nino12', 'nino3', 'nino4', 'nino34']].min().min()
max_value = df[['nino12', 'nino3', 'nino4', 'nino34']].max().max()

# Ajustar el rango del eje y
y_range = [min_value - 0.5, max_value + 0.5]

# Crear gráfico de líneas para cada columna de datos con ajuste del eje y
fig_line = px.line(df, x='Date', y=['nino12', 'nino3', 'nino4', 'nino34'],
                   labels={'value': 'Anomalía', 'variable': 'Series'},
                   title='Serie de Tiempo - Anomalías Niño')

fig_line.update_yaxes(range=y_range)

# Agregar línea horizontal roja punteada en 0.5
fig_line.add_shape(
    go.layout.Shape(
        type="line",
        x0=df['Date'].min(),
        y0=0.5,
        x1=df['Date'].max(),
        y1=0.5,
        line=dict(color="Red", width=2, dash="dash")
    )
)

# Agregar línea horizontal azul punteada en -0.5
fig_line.add_shape(
    go.layout.Shape(
        type="line",
        x0=df['Date'].min(),
        y0=-0.5,
        x1=df['Date'].max(),
        y1=-0.5,
        line=dict(color="Blue", width=2, dash="dash")
    )
)

# Crear gráfico de áreas para cada columna de datos con ajuste del eje y
fig_area = px.area(df, x='Date', y=['nino12', 'nino3', 'nino4', 'nino34'],
                   labels={'value': 'Anomalía', 'variable': 'Series'},
                   title='Serie de Tiempo - Anomalías Niño (Gráfico de Áreas)')

fig_area.update_yaxes(range=y_range)

# Agregar línea horizontal roja punteada en 0.5
fig_area.add_shape(
    go.layout.Shape(
        type="line",
        x0=df['Date'].min(),
        y0=0.5,
        x1=df['Date'].max(),
        y1=0.5,
        line=dict(color="Red", width=2, dash="dash")
    )
)

# Agregar línea horizontal azul punteada en -0.5
fig_area.add_shape(
    go.layout.Shape(
        type="line",
        x0=df['Date'].min(),
        y0=-0.5,
        x1=df['Date'].max(),
        y1=-0.5,
        line=dict(color="Blue", width=2, dash="dash")
    )
)

# Mostrar los gráficos
fig_line.show()


In [104]:
# Crear gráfico de líneas múltiples
fig = go.Figure()

for col in ['nino12', 'nino3', 'nino4', 'nino34']:
    fig.add_trace(go.Scatter(x=df['Date'], y=df[col], mode='lines', name=col))

# Agregar línea horizontal roja punteada en 0.5
fig.add_shape(
    go.layout.Shape(
        type="line",
        x0=df['Date'].min(),
        y0=0.5,
        x1=df['Date'].max(),
        y1=0.5,
        line=dict(color="Red", width=2, dash="dash")
    )
)

# Agregar línea horizontal azul punteada en -0.5
fig.add_shape(
    go.layout.Shape(
        type="line",
        x0=df['Date'].min(),
        y0=-0.5,
        x1=df['Date'].max(),
        y1=-0.5,
        line=dict(color="Blue", width=2, dash="dash")
    )
)

# Ajustar el rango del eje y
fig.update_yaxes(range=y_range)

# Agregar selector de rango
fig.update_layout(
    title='Serie de Tiempo - Anomalías Niño (Gráfico de Líneas con Selector de Rango)',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date"
    )
)

# Mostrar el gráfico
fig.show()

In [105]:
df.shape

(1853, 5)